In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.make'):
    shutil.rmtree('./file/3.make')

os.makedirs('./file/3.make')

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1

    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Part Code'] = input_['Part Code'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error, key

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                data = {'getArticleLinkedAllLinkingTargetManufacturer2': {'articleCountry': 'DE',
                                                                          'articleId': int(input_.loc[a, 'Part Code']),
                                                                          'country': 'DE',
                                                                          'countryGroupFlag': False,
                                                                          'linkingTargetType': 'VOLB',
                                                                          'provider': 40_000}}
                
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                             data=json.dumps(data),
                                             headers=get_header(**{'x-api-key': key}),
                                             #proxies=get_proxy(),
                                             timeout=(10, 10))
                            
                        if resp.status_code == 200:
                            json_ = resp.json()
                
                            if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                                while True:
                                    try:
                                        data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                                     'username': '186299u15',
                                                                     'password': 'APMTeDoW14'}}
                                        
                                        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                                             data=json.dumps(data),
                                                             headers=get_header(),
                                                             #proxies=get_proxy(),
                                                             timeout=(10, 10))
                                
                                        if resp.status_code == 200:
                                            json_ = resp.json()
                                
                                            if json_['status'] == 200:
                                                key = json_['apiKey']
                                                break
                                    except KeyboardInterrupt:
                                        break
                                    except:
                                        continue
                            elif json_['status'] == 200:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_make = json_['data']['array']

                # = = = = = = = = = = = = = = =

                if not list_make:
                    raise

                # = = = = = = = = = = = = = = =

                list_make_code = [make['manuId'] for make in list_make]

                # = = = = = = = = = = = = = = =

                list_make = [make['manuName'].strip() for make in list_make]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                        'Part Number': input_.loc[a, 'Part Number'],
                                        'Part Code': input_.loc[a, 'Part Code'],
                                        'Link Type': 'VOLB',
                                        'Make No': [i+1 for i in range(len(list_make))],
                                        'Make': list_make,
                                        'Make Code': list_make_code})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(50):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Make No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.make/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-make_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.make'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.make/{file}',
              f'''./file/3.make/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 859.176130
[剩余数量：1000] - [当前时间：16:41:09]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 825.175266
[剩余数量：999] - [当前时间：16:41:09]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 862.176346
[剩余数量：998] - [当前时间：16:41:09]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 863.176348
[剩余数量：997] - [当前时间：16:41:09]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 868.176526
[剩余数量：996] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 864.176358
[剩余数量：995] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 866.176406
[剩余数量：994] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 870.176531
[剩余数量：993] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 835.175425
[剩余数量：992] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 869.176530
[剩余数量：991] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 865.176368
[剩余数量：990] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 836.175426
[剩余数量：989] - [当前时间：16:41:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 873.176656
[剩

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4412.73it/s]

Done ~
